In [1]:
import mindspore.nn as nn
import mindspore.dataset as ds
import mindspore as ms
from mindspore import Tensor, Model, context
import mindspore.dataset.vision as vision
import mindspore.dataset.transforms as transforms
from mindspore.train.callback import LossMonitor
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# 设置使用 GPU
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

# Step 1: 加载并处理数据集
df = pd.read_csv("processed_dataset.csv")

# 将 'explicit' 列转换为数值型 (True/False 转换为 1/0)
df['explicit'] = df['explicit'].astype(int)

# 将类别特征 'track_genre' 做 One-Hot 编码
encoder = OneHotEncoder(sparse_output=False)
track_genre_encoded = encoder.fit_transform(df[['track_genre']])

# 将编码后的数据添加回数据框并移除原始的 'track_genre' 列
encoded_columns = encoder.get_feature_names_out(['track_genre'])
df_encoded = pd.DataFrame(track_genre_encoded, columns=encoded_columns)
df = pd.concat([df.drop(columns=['track_genre']), df_encoded], axis=1)

# 对类别特征 'key', 'mode', 'time_signature' 进行 One-Hot 编码
categorical_features = ['key', 'mode', 'time_signature']
categorical_encoder = OneHotEncoder(sparse_output=False, drop='first')
categorical_encoded = categorical_encoder.fit_transform(df[categorical_features])
categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=categorical_encoder.get_feature_names_out(categorical_features))

# 合并编码后的数据和数值特征，并移除原始的类别特征
df = pd.concat([df.drop(columns=categorical_features).reset_index(drop=True), categorical_encoded_df], axis=1)

# 标准化数值特征
numerical_features = df.select_dtypes(include=['float64', 'int64'])
scaler = StandardScaler()
df[numerical_features.columns] = scaler.fit_transform(numerical_features)

# 将标签转换为 float32 类型
df_encoded = df_encoded.astype(np.float32)

# 提取特征和标签
X = df.drop(columns=encoded_columns).astype(np.float32).values
y = df_encoded.values

# Step 2: 定义一个简单的 MLP 分类器
class MLPClassifier(nn.Cell):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLPClassifier, self).__init__()
        # 输入层到第一层隐藏层的全连接层
        self.fc1 = nn.Dense(input_size, hidden_sizes[0], weight_init="normal")
        # 激活函数 ReLU
        self.relu1 = nn.ReLU()
        # 第一层隐藏层到第二层隐藏层的全连接层
        self.fc2 = nn.Dense(hidden_sizes[0], hidden_sizes[1], weight_init="normal")
        # 激活函数 ReLU
        self.relu2 = nn.ReLU()
        # 第二层隐藏层到第三层隐藏层的全连接层
        self.fc3 = nn.Dense(hidden_sizes[1], hidden_sizes[2], weight_init="normal")
        # 激活函数 ReLU
        self.relu3 = nn.ReLU()
        # 第三层隐藏层到输出层的全连接层
        self.fc4 = nn.Dense(hidden_sizes[2], output_size, weight_init="normal")

    def construct(self, x):
        # 数据通过输入层进入第一层隐藏层
        x = self.fc1(x)
        # 使用 ReLU 激活函数
        x = self.relu1(x)
        # 数据通过第一层隐藏层进入第二层隐藏层
        x = self.fc2(x)
        # 使用 ReLU 激活函数
        x = self.relu2(x)
        # 数据通过第二层隐藏层进入第三层隐藏层
        x = self.fc3(x)
        # 使用 ReLU 激活函数
        x = self.relu3(x)
        # 数据通过第三层隐藏层进入输出层
        x = self.fc4(x)
        return x

# Step 3: 创建模型实例
input_size = X.shape[1]  # 输入层的特征数量
hidden_sizes = [100, 150, 100]  # 隐藏层中的神经元数量
output_size = y.shape[1]  # 输出层的神经元数量（类别数量）
model = MLPClassifier(input_size, hidden_sizes, output_size)

# Step 4: 定义超参数、损失函数和优化器
epochs = 10
batch_size = 32
learning_rate = 0.01

loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=False, reduction='mean')  # 损失函数
optimizer = nn.Adam(params=model.trainable_params(), learning_rate=learning_rate)  # 优化器

# Step 5: 割分训练集和测试集
# 将数据集割分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_dataset = ds.NumpySlicesDataset(data=(X_train, y_train), column_names=['features', 'labels'], shuffle=True).batch(batch_size)
test_dataset = ds.NumpySlicesDataset(data=(X_test, y_test), column_names=['features', 'labels'], shuffle=False).batch(batch_size)

# Step 6: 定义训练和测试函数
def forward_fn(data, label):
    logits = model(data)
    loss = loss_fn(logits, label)
    return loss, logits

grad_fn = ms.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

def train_step(data, label):
    (loss, _), grads = grad_fn(data, label)
    optimizer(grads)
    return loss

def train_loop(model, dataset, loss_fn):
    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)
        if batch % 100 == 0:
            loss_value = loss.asnumpy()
            print(f"loss: {loss_value:>7f}  [{batch:>3d}/{size:>3d}]")

def test_loop(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label.argmax(1)).asnumpy().sum()
    test_loss /= num_batches
    correct /= total
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# Step 7: 开始训练与测试
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model, train_dataset, loss_fn)
    test_loop(model, test_dataset, loss_fn)
print("Done!")

# Step 8: 使用模型进行推理
# 创建一个形状为 (1, input_size) 的全为 1 的形状作为输入
X_infer = Tensor(np.ones((1, input_size)), ms.float32)
# 使用模型进行前向传播，获取输出 logits
y_pred = model(X_infer)
# 打印预测结果
print(f"Predicted class probabilities: {y_pred}")

Epoch 1
-------------------------------


[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:49:12.108.472 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/3671574363.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:49:12.108.511 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/3671574363.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:49:12.108.526 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/3671574363.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:49:12.108.541 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/3671574363.py]


loss: 4.746007  [  0/2850]
loss: 4.547727  [100/2850]
loss: 4.449991  [200/2850]
loss: 4.089506  [300/2850]
loss: 4.335276  [400/2850]
loss: 4.309237  [500/2850]
loss: 3.839746  [600/2850]
loss: 3.850352  [700/2850]
loss: 3.904968  [800/2850]
loss: 3.871197  [900/2850]
loss: 4.159403  [1000/2850]
loss: 3.878630  [1100/2850]
loss: 3.923908  [1200/2850]
loss: 3.895950  [1300/2850]
loss: 3.775531  [1400/2850]
loss: 4.101347  [1500/2850]
loss: 3.804650  [1600/2850]
loss: 3.912532  [1700/2850]
loss: 3.883594  [1800/2850]
loss: 3.781496  [1900/2850]
loss: 3.562150  [2000/2850]
loss: 3.837533  [2100/2850]
loss: 3.751155  [2200/2850]
loss: 3.876537  [2300/2850]
loss: 3.853651  [2400/2850]
loss: 3.962660  [2500/2850]
loss: 3.743416  [2600/2850]
loss: 4.294083  [2700/2850]
loss: 3.395361  [2800/2850]


[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:49:21.574.062 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/3671574363.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:49:21.574.098 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/3671574363.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:49:23.584.541 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/3671574363.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:49:23.584.577 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/3671574363.py]


Test: 
 Accuracy: 12.6%, Avg loss: 3.706143 

Epoch 2
-------------------------------
loss: 3.351897  [  0/2850]
loss: 3.626379  [100/2850]
loss: 3.813746  [200/2850]
loss: 4.021282  [300/2850]
loss: 3.691102  [400/2850]
loss: 3.723602  [500/2850]
loss: 4.227628  [600/2850]
loss: 3.853584  [700/2850]
loss: 3.670685  [800/2850]
loss: 3.772881  [900/2850]
loss: 3.447707  [1000/2850]
loss: 3.802534  [1100/2850]
loss: 3.778628  [1200/2850]
loss: 3.520260  [1300/2850]
loss: 3.652048  [1400/2850]
loss: 3.700798  [1500/2850]
loss: 3.788813  [1600/2850]
loss: 3.941547  [1700/2850]
loss: 3.806073  [1800/2850]
loss: 3.653757  [1900/2850]
loss: 3.935611  [2000/2850]
loss: 3.879865  [2100/2850]
loss: 3.606355  [2200/2850]
loss: 3.574927  [2300/2850]
loss: 3.532648  [2400/2850]
loss: 3.498623  [2500/2850]
loss: 3.702773  [2600/2850]
loss: 3.145375  [2700/2850]
loss: 3.973557  [2800/2850]
Test: 
 Accuracy: 12.9%, Avg loss: 3.651599 

Epoch 3
-------------------------------
loss: 3.942368  [  0/2850]

[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:50:52.271.234 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/3671574363.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:50:52.271.273 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/3671574363.py]


In [2]:
import mindspore.nn as nn
import mindspore.dataset as ds
import mindspore as ms
from mindspore import Tensor, Model, context
import mindspore.dataset.vision as vision
import mindspore.dataset.transforms as transforms
from mindspore.train.callback import LossMonitor
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# 设置使用 GPU
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

# Step 1: 加载并处理数据集
df = pd.read_csv("processed_dataset.csv")

# 将 'explicit' 列转换为数值型 (True/False 转换为 1/0)
df['explicit'] = df['explicit'].astype(int)

# 将类别特征 'track_genre' 做 One-Hot 编码
encoder = OneHotEncoder(sparse_output=False)
track_genre_encoded = encoder.fit_transform(df[['track_genre']])

# 将编码后的数据添加回数据框并移除原始的 'track_genre' 列
encoded_columns = encoder.get_feature_names_out(['track_genre'])
df_encoded = pd.DataFrame(track_genre_encoded, columns=encoded_columns)
df = pd.concat([df.drop(columns=['track_genre']), df_encoded], axis=1)

# 对类别特征 'key', 'mode', 'time_signature' 进行 One-Hot 编码
categorical_features = ['key', 'mode', 'time_signature']
categorical_encoder = OneHotEncoder(sparse_output=False, drop='first')
categorical_encoded = categorical_encoder.fit_transform(df[categorical_features])
categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=categorical_encoder.get_feature_names_out(categorical_features))

# 合并编码后的数据和数值特征，并移除原始的类别特征
df = pd.concat([df.drop(columns=categorical_features).reset_index(drop=True), categorical_encoded_df], axis=1)

# 标准化数值特征
numerical_features = df.select_dtypes(include=['float64', 'int64'])
scaler = StandardScaler()
df[numerical_features.columns] = scaler.fit_transform(numerical_features)

# 将标签转换为 float32 类型
df_encoded = df_encoded.astype(np.float32)

# 提取特征和标签
X = df.drop(columns=encoded_columns).astype(np.float32).values
y = df_encoded.values

# Step 2: 定义一个简单的 MLP 分类器
class MLPClassifier(nn.Cell):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLPClassifier, self).__init__()
        # 输入层到第一层隐藏层的全连接层
        self.fc1 = nn.Dense(input_size, hidden_sizes[0], weight_init="normal")
        # 激活函数 ReLU
        self.relu1 = nn.ReLU()
        # 第一层隐藏层到第二层隐藏层的全连接层
        self.fc2 = nn.Dense(hidden_sizes[0], hidden_sizes[1], weight_init="normal")
        # 激活函数 ReLU
        self.relu2 = nn.ReLU()
        # 第二层隐藏层到第三层隐藏层的全连接层
        self.fc3 = nn.Dense(hidden_sizes[1], hidden_sizes[2], weight_init="normal")
        # 激活函数 ReLU
        self.relu3 = nn.ReLU()
        # 第三层隐藏层到输出层的全连接层
        self.fc4 = nn.Dense(hidden_sizes[2], output_size, weight_init="normal")

    def construct(self, x):
        # 数据通过输入层进入第一层隐藏层
        x = self.fc1(x)
        # 使用 ReLU 激活函数
        x = self.relu1(x)
        # 数据通过第一层隐藏层进入第二层隐藏层
        x = self.fc2(x)
        # 使用 ReLU 激活函数
        x = self.relu2(x)
        # 数据通过第二层隐藏层进入第三层隐藏层
        x = self.fc3(x)
        # 使用 ReLU 激活函数
        x = self.relu3(x)
        # 数据通过第三层隐藏层进入输出层
        x = self.fc4(x)
        return x

# Step 3: 创建模型实例
input_size = X.shape[1]  # 输入层的特征数量
hidden_sizes = [100, 150, 100]  # 隐藏层中的神经元数量
output_size = y.shape[1]  # 输出层的神经元数量（类别数量）
model = MLPClassifier(input_size, hidden_sizes, output_size)

# Step 4: 定义超参数、损失函数和优化器
epochs = 10
batch_size = 64
learning_rate = 0.01

loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=False, reduction='mean')  # 损失函数
optimizer = nn.Adam(params=model.trainable_params(), learning_rate=learning_rate)  # 优化器

# Step 5: 割分训练集和测试集
# 将数据集割分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_dataset = ds.NumpySlicesDataset(data=(X_train, y_train), column_names=['features', 'labels'], shuffle=True).batch(batch_size)
test_dataset = ds.NumpySlicesDataset(data=(X_test, y_test), column_names=['features', 'labels'], shuffle=False).batch(batch_size)

# Step 6: 定义训练和测试函数
def forward_fn(data, label):
    logits = model(data)
    loss = loss_fn(logits, label)
    return loss, logits

grad_fn = ms.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

def train_step(data, label):
    (loss, _), grads = grad_fn(data, label)
    optimizer(grads)
    return loss

def train_loop(model, dataset, loss_fn):
    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)
        if batch % 100 == 0:
            loss_value = loss.asnumpy()
            print(f"loss: {loss_value:>7f}  [{batch:>3d}/{size:>3d}]")

def test_loop(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label.argmax(1)).asnumpy().sum()
    test_loss /= num_batches
    correct /= total
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# Step 7: 开始训练与测试
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model, train_dataset, loss_fn)
    test_loop(model, test_dataset, loss_fn)
print("Done!")

# Step 8: 使用模型进行推理
# 创建一个形状为 (1, input_size) 的全为 1 的形状作为输入
X_infer = Tensor(np.ones((1, input_size)), ms.float32)
# 使用模型进行前向传播，获取输出 logits
y_pred = model(X_infer)
# 打印预测结果
print(f"Predicted class probabilities: {y_pred}")

Epoch 1
-------------------------------


[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:50:55.526.792 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2178181729.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:50:55.526.825 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2178181729.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:50:55.526.839 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2178181729.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:50:55.526.855 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2178181729.py]


loss: 4.751536  [  0/1425]
loss: 4.134300  [100/1425]
loss: 3.936512  [200/1425]
loss: 3.930590  [300/1425]
loss: 3.666158  [400/1425]
loss: 3.891729  [500/1425]
loss: 3.787324  [600/1425]
loss: 3.812965  [700/1425]
loss: 3.572407  [800/1425]
loss: 3.691097  [900/1425]
loss: 3.393146  [1000/1425]
loss: 3.708950  [1100/1425]
loss: 3.594028  [1200/1425]
loss: 3.922259  [1300/1425]
loss: 3.497707  [1400/1425]


[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:50:59.823.219 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2178181729.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:50:59.823.253 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2178181729.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:51:01.915.933 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2178181729.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:51:01.915.973 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2178181729.py]


Test: 
 Accuracy: 14.1%, Avg loss: 3.585320 

Epoch 2
-------------------------------
loss: 3.553270  [  0/1425]
loss: 3.485363  [100/1425]
loss: 3.498821  [200/1425]
loss: 3.337526  [300/1425]
loss: 3.674687  [400/1425]
loss: 3.675710  [500/1425]
loss: 3.414034  [600/1425]
loss: 3.751576  [700/1425]
loss: 3.639056  [800/1425]
loss: 3.541929  [900/1425]
loss: 3.061131  [1000/1425]
loss: 3.483802  [1100/1425]
loss: 3.263660  [1200/1425]
loss: 3.395093  [1300/1425]
loss: 3.764843  [1400/1425]
Test: 
 Accuracy: 15.3%, Avg loss: 3.536888 

Epoch 3
-------------------------------
loss: 3.617478  [  0/1425]
loss: 3.618400  [100/1425]
loss: 3.120779  [200/1425]
loss: 3.831576  [300/1425]
loss: 3.643434  [400/1425]
loss: 3.723781  [500/1425]
loss: 3.495367  [600/1425]
loss: 3.531024  [700/1425]
loss: 3.496921  [800/1425]
loss: 3.438411  [900/1425]
loss: 3.450445  [1000/1425]
loss: 3.555105  [1100/1425]
loss: 3.414358  [1200/1425]
loss: 3.228686  [1300/1425]
loss: 3.481063  [1400/1425]
Test: 
 

[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:52:06.900.380 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2178181729.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:52:06.900.426 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2178181729.py]


In [3]:
import mindspore.nn as nn
import mindspore.dataset as ds
import mindspore as ms
from mindspore import Tensor, Model, context
import mindspore.dataset.vision as vision
import mindspore.dataset.transforms as transforms
from mindspore.train.callback import LossMonitor
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# 设置使用 GPU
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

# Step 1: 加载并处理数据集
df = pd.read_csv("processed_dataset.csv")

# 将 'explicit' 列转换为数值型 (True/False 转换为 1/0)
df['explicit'] = df['explicit'].astype(int)

# 将类别特征 'track_genre' 做 One-Hot 编码
encoder = OneHotEncoder(sparse_output=False)
track_genre_encoded = encoder.fit_transform(df[['track_genre']])

# 将编码后的数据添加回数据框并移除原始的 'track_genre' 列
encoded_columns = encoder.get_feature_names_out(['track_genre'])
df_encoded = pd.DataFrame(track_genre_encoded, columns=encoded_columns)
df = pd.concat([df.drop(columns=['track_genre']), df_encoded], axis=1)

# 对类别特征 'key', 'mode', 'time_signature' 进行 One-Hot 编码
categorical_features = ['key', 'mode', 'time_signature']
categorical_encoder = OneHotEncoder(sparse_output=False, drop='first')
categorical_encoded = categorical_encoder.fit_transform(df[categorical_features])
categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=categorical_encoder.get_feature_names_out(categorical_features))

# 合并编码后的数据和数值特征，并移除原始的类别特征
df = pd.concat([df.drop(columns=categorical_features).reset_index(drop=True), categorical_encoded_df], axis=1)

# 标准化数值特征
numerical_features = df.select_dtypes(include=['float64', 'int64'])
scaler = StandardScaler()
df[numerical_features.columns] = scaler.fit_transform(numerical_features)

# 将标签转换为 float32 类型
df_encoded = df_encoded.astype(np.float32)

# 提取特征和标签
X = df.drop(columns=encoded_columns).astype(np.float32).values
y = df_encoded.values

# Step 2: 定义一个简单的 MLP 分类器
class MLPClassifier(nn.Cell):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(MLPClassifier, self).__init__()
        # 输入层到第一层隐藏层的全连接层
        self.fc1 = nn.Dense(input_size, hidden_sizes[0], weight_init="normal")
        # 激活函数 ReLU
        self.relu1 = nn.ReLU()
        # 第一层隐藏层到第二层隐藏层的全连接层
        self.fc2 = nn.Dense(hidden_sizes[0], hidden_sizes[1], weight_init="normal")
        # 激活函数 ReLU
        self.relu2 = nn.ReLU()
        # 第二层隐藏层到第三层隐藏层的全连接层
        self.fc3 = nn.Dense(hidden_sizes[1], hidden_sizes[2], weight_init="normal")
        # 激活函数 ReLU
        self.relu3 = nn.ReLU()
        # 第三层隐藏层到输出层的全连接层
        self.fc4 = nn.Dense(hidden_sizes[2], output_size, weight_init="normal")

    def construct(self, x):
        # 数据通过输入层进入第一层隐藏层
        x = self.fc1(x)
        # 使用 ReLU 激活函数
        x = self.relu1(x)
        # 数据通过第一层隐藏层进入第二层隐藏层
        x = self.fc2(x)
        # 使用 ReLU 激活函数
        x = self.relu2(x)
        # 数据通过第二层隐藏层进入第三层隐藏层
        x = self.fc3(x)
        # 使用 ReLU 激活函数
        x = self.relu3(x)
        # 数据通过第三层隐藏层进入输出层
        x = self.fc4(x)
        return x

# Step 3: 创建模型实例
input_size = X.shape[1]  # 输入层的特征数量
hidden_sizes = [100, 150, 100]  # 隐藏层中的神经元数量
output_size = y.shape[1]  # 输出层的神经元数量（类别数量）
model = MLPClassifier(input_size, hidden_sizes, output_size)

# Step 4: 定义超参数、损失函数和优化器
epochs = 10
batch_size = 128
learning_rate = 0.01

loss_fn = nn.SoftmaxCrossEntropyWithLogits(sparse=False, reduction='mean')  # 损失函数
optimizer = nn.Adam(params=model.trainable_params(), learning_rate=learning_rate)  # 优化器

# Step 5: 割分训练集和测试集
# 将数据集割分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_dataset = ds.NumpySlicesDataset(data=(X_train, y_train), column_names=['features', 'labels'], shuffle=True).batch(batch_size)
test_dataset = ds.NumpySlicesDataset(data=(X_test, y_test), column_names=['features', 'labels'], shuffle=False).batch(batch_size)

# Step 6: 定义训练和测试函数
def forward_fn(data, label):
    logits = model(data)
    loss = loss_fn(logits, label)
    return loss, logits

grad_fn = ms.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

def train_step(data, label):
    (loss, _), grads = grad_fn(data, label)
    optimizer(grads)
    return loss

def train_loop(model, dataset, loss_fn):
    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)
        if batch % 100 == 0:
            loss_value = loss.asnumpy()
            print(f"loss: {loss_value:>7f}  [{batch:>3d}/{size:>3d}]")

def test_loop(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label.argmax(1)).asnumpy().sum()
    test_loss /= num_batches
    correct /= total
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# Step 7: 开始训练与测试
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(model, train_dataset, loss_fn)
    test_loop(model, test_dataset, loss_fn)
print("Done!")

# Step 8: 使用模型进行推理
# 创建一个形状为 (1, input_size) 的全为 1 的形状作为输入
X_infer = Tensor(np.ones((1, input_size)), ms.float32)
# 使用模型进行前向传播，获取输出 logits
y_pred = model(X_infer)
# 打印预测结果
print(f"Predicted class probabilities: {y_pred}")

Epoch 1
-------------------------------


[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:52:10.167.475 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2105147425.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:52:10.167.513 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2105147425.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:52:10.167.528 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2105147425.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:52:10.167.543 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2105147425.py]


loss: 4.740317  [  0/713]
loss: 4.079561  [100/713]
loss: 3.894694  [200/713]
loss: 3.885770  [300/713]
loss: 3.838417  [400/713]
loss: 3.582614  [500/713]
loss: 3.318523  [600/713]
loss: 3.677637  [700/713]


[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:52:13.864.293 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2105147425.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:52:13.864.326 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2105147425.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:52:13.864.341 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2105147425.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:52:13.864.356 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2105147425.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:52:13.967.896 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2105147425.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:52:13.967.930 [mindspore/core/utils/f

Test: 
 Accuracy: 14.4%, Avg loss: 3.556221 

Epoch 2
-------------------------------
loss: 3.610327  [  0/713]
loss: 3.572551  [100/713]
loss: 3.480569  [200/713]
loss: 3.614778  [300/713]
loss: 3.466695  [400/713]
loss: 3.474248  [500/713]
loss: 3.314315  [600/713]
loss: 3.444158  [700/713]
Test: 
 Accuracy: 15.6%, Avg loss: 3.484475 

Epoch 3
-------------------------------
loss: 3.296033  [  0/713]
loss: 3.335403  [100/713]
loss: 3.521816  [200/713]
loss: 3.398899  [300/713]
loss: 3.269041  [400/713]
loss: 3.241086  [500/713]
loss: 3.161129  [600/713]
loss: 3.409530  [700/713]
Test: 
 Accuracy: 17.1%, Avg loss: 3.435735 

Epoch 4
-------------------------------
loss: 3.524062  [  0/713]
loss: 3.255773  [100/713]
loss: 3.386379  [200/713]
loss: 3.249185  [300/713]
loss: 3.228599  [400/713]
loss: 3.709189  [500/713]
loss: 3.585850  [600/713]
loss: 3.296715  [700/713]
Test: 
 Accuracy: 16.7%, Avg loss: 3.442930 

Epoch 5
-------------------------------
loss: 3.218876  [  0/713]
loss: 

[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:53:01.749.234 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2105147425.py]
[ERROR] CORE(1760,7f2fed8464c0,python):2024-10-14-13:53:01.749.277 [mindspore/core/utils/file_utils.cc:253] GetRealPath] Get realpath failed, path[/tmp/ipykernel_1760/2105147425.py]
